In [90]:
import json
import numpy as np
import random

fever_path = "/private/home/xwhan/data/fever/retrieval/"

dev = [json.loads(l) for l in open(fever_path + "dev.txt").readlines()]
multi_dev = []
single_dev = []
all_evidence_lens = [] # for multi evidence
random.shuffle(dev)
all_claim_lens = []
for item in dev:
    evidence_lens = []
    all_claim_lens.append(len(item["claim"].split()))
    
    for chain in item["evidence"]:
        if len(chain) > 1:
#             evidence_lens.append(len(chain))
            chain_titles = set([p["title"] for p in chain])
            evidence_lens.append(len(chain_titles))  
#             print(item["claim"])
#             print(chain)
#             assert False
        else:
            evidence_lens.append(1)
    multi_count = np.sum([int(c > 1) for c in evidence_lens])
    
    if multi_count == len(evidence_lens):
        multi_dev.append(item)
        all_evidence_lens += evidence_lens
    else:
        single_dev.append(item)
        
print(len(multi_dev))
print(len(single_dev))
with open("/private/home/xwhan/data/fever/retrieval/dev_multi_evidence_compact.txt", "w") as g:
    for l in multi_dev:
        g.write(json.dumps(l) + "\n")
# with open("/private/home/xwhan/data/fever/retrieval/dev_single_evidence.txt", "w") as g:
#     for l in single_dev:
#         g.write(json.dumps(l) + "\n")
# with open("/private/home/xwhan/data/fever/retrieval/dev_all.txt", "w") as g:
#     for l in single_dev + multi_dev:
#         g.write(json.dumps(l) + "\n")

1059
12273


In [81]:
# baseline retrieval for single/multihop subsets

import unicodedata
def normalize(text):
    """Resolve different type of unicode encodings."""
    return unicodedata.normalize('NFD', text)

el_results = [json.loads(l) for l in open("/private/home/xwhan/data/fever/retrieval/dev.ensembles.s10.jsonl").readlines()]
id2el_docs = {_["id"]:_["predicted_pages"] for _ in el_results}

dense_multi_results = [json.loads(l) for l in open("/private/home/xwhan/data/fever/retrieval/dense_fever_b1_20_k20.json").readlines()] 

single_gold = {_["id"]:_ for _ in single_dev}
multi_gold = {_["id"]:_ for _ in multi_dev}
all_gold = {_["id"]:_ for _ in multi_dev + single_dev}

subset = multi_gold
precs, recalls = [], []
doc_count = []
dense_docs = []
for item in dense_multi_results:
    if item["id"] in subset:
#         pred = set(item["predicted_pages"])
        retrieved_chains = item["candidate_chains"] 
        pred = []
        for chain in retrieved_chains:
            for p in chain:
                if normalize(p[0]) not in pred:
                    pred.append(normalize(p[0]))
        pred = pred[:2]
#         pred = [_["title"] for _ in item["topk"][:1]]
        
        pred = set(pred)
#         el_pred = id2el_docs[item["id"]]
#         el_count = 0
#         for title in el_pred:
#             if title not in pred:
#                 pred.add(title)
#                 el_count +=1
#                 if el_count == 2:
#                     break
        pred = list(pred)
    
        dense_docs.append({
            "claim": item["claim"],
            "id": item["id"],
            "predicted_pages": list(pred)
        })
        
        doc_count.append(len(pred))
    
        gold_docs = set()
        recall = 0
        for chain in subset[item["id"]]["evidence"]:
            chain_titles = set([normalize(p["title"]) for p in chain])
            for t in chain_titles: gold_docs.add(t)
            chain_covered = [int(t in pred) for t in chain_titles]
            if np.sum(chain_covered) == len(chain_titles):
                recall = 1
                break
                
        if len(gold_docs) > 0:
            if len(pred) == 0:
                prec = 0
            else:
                prec = np.mean([int(doc in gold_docs) for doc in pred])
                
        precs.append(prec)
        recalls.append(recall)
        
print(len(precs))
print(np.mean(doc_count))
pr, rec = np.mean(precs), np.mean(recalls)
print(pr, rec, 2.0 * pr * rec / (pr + rec))

# with open("/private/home/xwhan/data/fever/retrieval/dense_wiki_pages_top2.jsonl", "w") as g:
#     for _ in dense_docs:
#         g.write(json.dumps(_) + "\n")

1741
2.0
0.5835726593911545 0.5835726593911545 0.5835726593911545


In [95]:
# inspect the FEVER results
dev_all_results = [json.loads(l) for l in open("/private/home/xwhan/code/Transformer-XH/data/fever_dev_graph.json").readlines()]
subset = multi_gold
precs, recalls = [], []
doc_count = []

for item in dev_all_results:
    if item["qid"] in subset:
        
        pred = [_["name"] for _ in item["node"]]
        pred = set(pred)
        pred = list(pred)
        
        doc_count.append(len(pred))
    
        gold_docs = set()
        recall = 0
        for chain in subset[item["qid"]]["evidence"]:
            chain_titles = set([normalize(p["title"]) for p in chain])
            for t in chain_titles: gold_docs.add(t)
            chain_covered = [int(t in pred) for t in chain_titles]
            if np.sum(chain_covered) == len(chain_titles):
                recall = 1
                break
                
        if len(gold_docs) > 0:
            if len(pred) == 0:
                prec = 0
            else:
                prec = np.mean([int(doc in gold_docs) for doc in pred])
                
        precs.append(prec)
        recalls.append(recall)
        
print(len(precs))
print(np.mean(doc_count))
pr, rec = np.mean(precs), np.mean(recalls)
print(pr, rec, 2.0 * pr * rec / (pr + rec))

1741
2.9959793222286044
0.6764680633362424 1.0 0.8070157471297632


In [40]:
# passage retrieval evaluation 
def fever_retrieval_eval(results, topk=5):
    
    precs, recalls = [], []
    for item in results:
        gold = item["correct_normalized"]
        pred = item["bm25_topk"][:topk]
        
        if len(gold) > 0:
            prec = np.mean([int(doc in gold) for doc in pred])
        else:
            prec = 1
        recall = 0
        for chain in item["evidence"]:
            chain_titles = set([normalize(p["title"]) for p in chain])
            chain_covered = [int(t in pred) for t in chain_titles]
            if np.sum(chain_covered) == len(chain_titles):
                recall = 1
                break
        precs.append(prec)
        recalls.append(recall)
    pr, rec = np.mean(precs), np.mean(recalls)
    return pr, rec, 2.0 * pr * rec / (pr + rec)

In [42]:
tfidf_results = [json.loads(l) for l in open("/private/home/xwhan/data/fever/retrieval/multi_dev_tfidf.txt").readlines()]
print(fever_retrieval_eval(tfidf_results, 10))

(0.12268811028144745, 0.5020103388856979, 0.19718537768662206)


In [244]:
# phrase_matching_results = [json.loads(l) for l in open("/private/home/xwhan/data/fever/retrieval/all_dev.json").readlines()]
# phrase_matching_results = [json.loads(l.decode('utf-8').strip('\r\n')) for l in open("/private/home/xwhan/data/fever/retrieval/dev.ensembles.s10.jsonl").readlines()]
phrase_matching_results = [json.loads(l) for l in open("/private/home/xwhan/code/Transformer-XH/data/fever_dev_graph.json").readlines()]
# for _ in phrase_matching_results:
#     _["id"] = _["qid"]

phrase_matching_results = [_ for _ in phrase_matching_results if _["id"] in multihop_ids]
# phrase_matching_results = [json.loads(l) for l in open("/private/home/xwhan/data/fever/retrieval/esim_mhop_dev.json").readlines()]

# json.dump(phrase_matching_results, open("/private/home/xwhan/data/fever/retrieval/dev_el_wiki_pages.jsonl", "w"))

print(len(phrase_matching_results))

tfidf_results = [json.loads(l) for l in open("/private/home/xwhan/data/fever/retrieval/multi_dev_tfidf.txt").readlines()]
print(len(tfidf_results))
pred_lens = []
def fever_retrieval_eval_phrase(tfidf_results, phrase_results, topk=5):
    id2gold = {_["id"]:_["correct_normalized"] for _ in tfidf_results}
    id2gold_evidence = {_["id"]:_["evidence"] for _ in tfidf_results}
    precs, recalls = [], []
    for item in phrase_results:
        gold = id2gold[item["id"]]
#         print(gold)
        retrieved_evidence = item["evidence"] 
        pred = []
        for e in retrieved_evidence:
            pred.append(normalize(e[0]))
        
#         pred = item["predicted_pages"] + item["wiki_results"]
#         pred = item["wiki_results"]
#         pred = item["predicted_pages"]
        
#         pred = []
#         for n in item["node"]:
#             pred.append(n["name"])
        
        pred = list(set(pred))
        pred_lens.append(len(pred))
        
        if len(gold) > 0:
            if len(pred) == 0:
                prec = 0
            else:
                prec = np.mean([int(doc in gold) for doc in pred])
        else:
            prec = 1
        recall = 0
        for chain in id2gold_evidence[item["id"]]:
            chain_titles = set([normalize(p["title"]) for p in chain])
            chain_covered = [int(t in pred) for t in chain_titles]
            if np.sum(chain_covered) == len(chain_titles):
                recall = 1
                break
        precs.append(prec)
        recalls.append(recall)
    
    pr, rec = np.mean(precs), np.mean(recalls)
    return pr, rec, 2.0 * pr * rec / (pr + rec)
print(fever_retrieval_eval_phrase(tfidf_results, phrase_matching_results, topk=5))

1741
1741
(0.6223243346735593, 0.46927053417576103, 0.5350675077296432)


In [181]:
np.mean(pred_lens)

2.1039632395175185

In [153]:
test_with_all = [json.loads(l) for l in open("/private/home/xwhan/data/fever/retrieval/all_dev.json").readlines()]
title_count = []
for item in test_with_all:
    titles = set()
    for e in item["evidence"]:
        titles.add(e[0])
    title_count.append(len(titles))
np.sum(np.array(title_count) > 7) / len(title_count)

0.12426242624262426

In [154]:
test_with_all = [json.loads(l) for l in open("/private/home/xwhan/data/fever/retrieval/all_test.json").readlines()]
title_count = []
for item in test_with_all:
    titles = set()
    for e in item["evidence"]:
        titles.add(e[0])
    title_count.append(len(titles))
np.sum(np.array(title_count) > 7) / len(title_count)

0.11321132113211321

In [91]:
# build dense final prediction for evaluation
final_pred = [json.loads(l) for l in open("/private/home/xwhan/code/KernelGAT/kgat/output/dense_bert_dev_mhop_top4.json").readlines()]
final_retrieval = [json.loads(l) for l in open("/private/home/xwhan/code/KernelGAT/data/bert_dense_top4_mhop_sents.json").readlines()]
all_dev_gold = [json.loads(l) for l in open("/private/home/xwhan/data/fever/shared_task_dev.jsonl").readlines()]
id2gold = {_["id"]:_ for _ in all_dev_gold}

print(len(final_retrieval))
final = []
for pred, retrieval in zip(final_pred, final_retrieval):
    assert pred["id"] == retrieval["id"]
    final.append({
        "id": pred["id"],
        "label": id2gold[pred["id"]]["label"],
        "evidence": id2gold[pred["id"]]["evidence"],
        "predicted_label": pred["predicted_label"],
        "predicted_evidence": [[normalize(e[0]), int(e[1])] for e in retrieval["evidence"][:5]]
    })

print(len(final))
with open("/private/home/xwhan/data/fever/results/dense_top4_mhop_dev.json", "w") as g:
    for l in final:
        g.write(json.dumps(l) + "\n")

1960
1960


In [92]:
# build EL final prediction for evaluation
final_pred = [json.loads(l) for l in open("/private/home/xwhan/code/KernelGAT/kgat/output/el_bert_dev_mhop.json").readlines()]
final_retrieval = [json.loads(l) for l in open("/private/home/xwhan/code/KernelGAT/data/bert_dev_multi_el.json").readlines()]
all_dev_gold = [json.loads(l) for l in open("/private/home/xwhan/data/fever/shared_task_dev.jsonl").readlines()]
id2gold = {_["id"]:_ for _ in all_dev_gold}

print(len(final_retrieval))
final = []
for pred, retrieval in zip(final_pred, final_retrieval):
    assert pred["id"] == retrieval["id"]
    final.append({
        "id": pred["id"],
        "label": id2gold[pred["id"]]["label"],
        "evidence": id2gold[pred["id"]]["evidence"],
        "predicted_label": pred["predicted_label"],
        "predicted_evidence": [[normalize(e[0]), int(e[1])] for e in retrieval["evidence"][:5]]
    })

print(len(final))
with open("/private/home/xwhan/data/fever/results/el_mhop_dev.json", "w") as g:
    for l in final:
        g.write(json.dumps(l) + "\n")

1960
1960


In [93]:
final_pred = [json.loads(l) for l in open("/private/home/xwhan/code/KernelGAT/kgat/output/esim_mhop_dev.json").readlines()]
final_retrieval = [json.loads(l) for l in open("/private/home/xwhan/data/fever/retrieval/esim_mhop_dev.json").readlines()]
all_dev_gold = [json.loads(l) for l in open("/private/home/xwhan/data/fever/shared_task_dev.jsonl").readlines()]
id2gold = {_["id"]:_ for _ in all_dev_gold}

final = []
for pred, retrieval in zip(final_pred, final_retrieval):
    assert pred["id"] == retrieval["id"]
    final.append({
        "id": pred["id"],
        "label": id2gold[pred["id"]]["label"],
        "evidence": id2gold[pred["id"]]["evidence"],
        "predicted_label": pred["predicted_label"],
        "predicted_evidence": [[e[0], int(e[1])] for e in retrieval["evidence"][:5]]
    })

print(len(final))
with open("/private/home/xwhan/data/fever/results/esim_mhop_dev.json", "w") as g:
    for l in final:
        g.write(json.dumps(l) + "\n")

1960
